## Visualisierung der freien Schwingungen des Einmassenschwingers
In diesem Notebook visualisieren wir die freien Schwingungen des Einmassenschwingers, d.h. für den Fall ohne Anregung. Der Einmassenschwinger wird also an einer Anfangsposition $x_0$ mit einer Anfangsgeschwindigkeit $v_0$ gestartet und ist dann sich selbst überlassen. Die Bewegungsgleichung des Einmassenschwingers lautet 

$$
\ddot{x} + 2 D \omega_0 \dot{x} + \omega_0^2 x = 0 \, .
$$

Die Lösung dieser Bewegungsgleichung für den Fall unterkritischer Dämpfung, d.h. $0\leq D<1$ ist die sogenannte homogene Lösung $x_h$ (homogen, da die rechte Seite der Differentialgleichung $0$ ist) und lautet 

$$
x_h(t) = e^{-D \omega_0 t} (A \cos(\omega_d t) + B \sin(\omega_d t)) \, .
$$

Die beiden Integrationskonstanten $A$, $B$ sind hier noch unbekannt. Um die Lösung eindeutig zu machen, werden $A$ und $B$ an die Anfangsbedingungen $x_h(t=0)=x_0$ und $\dot{x}_h(t=0)=v_0$ angepasst. 

### Importieren der Bibliotheken

In [10]:
import numpy as np
from sympy import *
from sympy.physics.vector import *
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from scipy.integrate import solve_ivp
from matplotlib import colors as mcolors
from ipywidgets import interactive, Layout

### Aufstellen der homogenenen Lösung
Wir definieren zunächst die symbolischen Variablen `D`, `omega0`, `A`, `B` und `t`. Danach stellen wir den symbolischen Ausdruck für `omega_d` und im Anschluss daran `xh` auf.

In [11]:
D, omega0, A, B, t = symbols('D, omega0, A, B, t')

omega_d = omega0*sqrt(1-D**2)
xh = exp(-D*omega0*t)*(A*cos(omega_d*t) + B*sin(omega_d*t))
xh

(A*cos(omega0*t*sqrt(1 - D**2)) + B*sin(omega0*t*sqrt(1 - D**2)))*exp(-D*omega0*t)

### Anpassen der Integrationskonstanten an Anfangsbedingungen
Um die Integrationskonstanten an die Anfangsbedingungen anzupassen, verwerden wir den `solve`-Befehl. Zuerst definieren wir hierfür die beiden neuen symbolischen Variablen `x0`, `v0`. Danach können die Anfangsbedingungen als `Eq`-Objekte definiert werden und mit `solve` nach `A`, `B` aufgelöst werden.

In [12]:
x0, v0 = symbols('x0, v0')

AB1 = Eq(xh.subs(t,0), x0)
AB2 = Eq(diff(xh,t).subs(t,0), v0)

intKonstanten = solve([AB1,AB2],[A,B])

display(intKonstanten[A], intKonstanten[B])

x0

D*x0/sqrt(1 - D**2) + v0/(omega0*sqrt(1 - D**2))

### Konvertierung der symbolischen Funktion `xh` in numerische Funktion `xh_num`
Wir setzen nun die gefundenen Ausdrücke für die Integrationskonstanten und die gewünschten Anfangsbedingungen mit `subs` in `xh` ein und konvertieren daraufhin den symbolischen Ausdruck `xh` mit `lambdify` in eine numerische Funktion, die wir zur Erstellung von Grafiken gebrauchen werden.

In [13]:
ABen = {x0: 1, v0: 0}
xh_num = lambdify([D,omega0,t], xh.subs(intKonstanten).subs(ABen))
xh_num(.5,1,np.array([1,2,3]))

array([ 0.65970015,  0.15057437, -0.12435477])

### Interaktive Grafik
Zu guter Letzt stellen wir die homogene Lösung $x_h$ mit unserer numerischen Funktion `xh_num` grafisch dar und gewährleisten eine gewisse Interaktivität mit je einem Schieberegler für die Eigenkreisfrequenz $\omega_0$ und den Dämpfungsgrad $D$.

In [9]:
# Zeit, t
tsteps = np.linspace(0,20,1000)
    
def update(omega0,D):        
    plt.figure(figsize=(9,6))
    plt.plot(tsteps,xh_num(D,omega0,tsteps))
    plt.xlim(*tsteps[[0,-1]])
    plt.ylim([-1.2,1.2])
    plt.xlabel('$t$ in s',size=15)
    plt.ylabel('$x$ in m',size=15)
    plt.grid(True)
    plt.show()
    

interactive_plot = interactive(update, omega0=(1,10,.01), D=(0,.99,.01))
widget1 = interactive_plot.children[0]
widget2 = interactive_plot.children[1]
widget1.layout = Layout(width='400px', align_items='center')
widget2.layout = Layout(width='400px', align_items='center')
widget1.description = '$\omega_0$ [rad/s]'
widget2.description = '$D$ [1]'
#widget1.continuous_update = True
#widget2.continuous_update = True
output = interactive_plot.children[-1]
interactive_plot.layout = Layout(width='570px',align_items='center')
interactive_plot

interactive(children=(FloatSlider(value=5.0, description='$\\omega_0$ [rad/s]', layout=Layout(align_items='cen…